In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/cs-ioc5008-hw1/sameple_submission.csv
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0068.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0095.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0031.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0054.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0106.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0094.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0044.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0025.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0043.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0123.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0080.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0117.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/office/image_0042.jpg

/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0238.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0076.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0234.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0073.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0204.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0018.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0022.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0113.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0038.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0006.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0229.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0110.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/forest/image_0098.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/

/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0149.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0298.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0543.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0722.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0370.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0221.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0832.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0553.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0068.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0095.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0991.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0031.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0268.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0491.jpg
/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test/image_0590.jpg
/kaggle/input/cs-ioc5008-

In [2]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

print("Import OK")

Using TensorFlow backend.


Import OK


In [19]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

In [20]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024, activation='relu')(x) #dense layer 2
x = Dense(512, activation='relu')(x) #dense layer 3
x = Dense(256, activation='relu')(x) #dense layer 4
preds = Dense(13, activation='softmax')(x) #final layer with softmax activation

In [25]:
model2 = Model(inputs=model.input, outputs=preds)

In [26]:
for layer in model2.layers[:20]:
    layer.trainable=False
for layer in model2.layers[20:]:
    layer.trainable=True

In [89]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
                                                 target_size=(250,250),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2819 images belonging to 13 classes.


In [90]:
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model2.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=15)

Epoch 1/15
88/88 [==============================] - 10s 112ms/step - loss: 0.1229 - accuracy: 0.9706
Epoch 2/15
88/88 [==============================] - 9s 107ms/step - loss: 0.0559 - accuracy: 0.9853
Epoch 3/15
88/88 [==============================] - 9s 108ms/step - loss: 0.1208 - accuracy: 0.9763
Epoch 4/15
88/88 [==============================] - 9s 107ms/step - loss: 0.0528 - accuracy: 0.9867
Epoch 5/15
88/88 [==============================] - 9s 108ms/step - loss: 0.0605 - accuracy: 0.9824
Epoch 6/15
88/88 [==============================] - 9s 107ms/step - loss: 0.0978 - accuracy: 0.9770
Epoch 7/15
88/88 [==============================] - 9s 108ms/step - loss: 0.0514 - accuracy: 0.9892
Epoch 8/15
88/88 [==============================] - 10s 109ms/step - loss: 0.0585 - accuracy: 0.9867
Epoch 9/15
88/88 [==============================] - 10s 108ms/step - loss: 0.1415 - accuracy: 0.9785
Epoch 10/15
88/88 [==============================] - 10s 108ms/step - loss: 0.1166 - accuracy: 0.

In [91]:
test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [92]:
def processing(image_data):
    #img = [data[123], data[1054], data[156]]
    #print("Original size:", img[1].shape)
    #print(training_label[1054])
    #original = img[1]
    #display_one(original)
    
    height = 250
    width = 250
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

In [77]:
print(test_data_reshape[0].shape)

(250, 250, 3)


In [97]:
#for i in range(len(test_data_reshape)):
#    test_data_reshape[i] = (test_data_reshape[i] - 128.0) / 128.0
    
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 250, 250, 3)
    
scene_detail = ["bedroom", "coast", "forest", "highway", "insidecity", "kitchen", "livingroom", "mountain", "office", "opencountry", "street", "suburb", "tallbuilding"]

print("id,label")

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    answer = model2.predict(test_data_reshape[i])
    print(name+","+scene_detail[np.argmax(answer)])


id,label
image_0149,forest
image_0298,highway
image_0543,suburb
image_0722,kitchen
image_0370,kitchen
image_0221,opencountry
image_0832,suburb
image_0553,tallbuilding
image_0068,kitchen
image_0095,livingroom
image_0991,opencountry
image_0031,opencountry
image_0268,opencountry
image_0491,opencountry
image_0590,street
image_0158,bedroom
image_0355,insidecity
image_0169,kitchen
image_0431,highway
image_0200,suburb
image_0054,suburb
image_0880,highway
image_0222,suburb
image_0723,bedroom
image_0828,livingroom
image_0106,mountain
image_0287,opencountry
image_0346,suburb
image_0266,livingroom
image_1020,highway
image_0611,bedroom
image_1009,tallbuilding
image_0588,insidecity
image_0164,opencountry
image_0243,tallbuilding
image_0094,opencountry
image_0847,bedroom
image_0701,highway
image_0468,street
image_0044,livingroom
image_0864,suburb
image_0968,mountain
image_0025,highway
image_0043,livingroom
image_0123,coast
image_0873,mountain
image_0999,office
image_0080,mountain
image_0117,office
im

image_0082,livingroom
image_0261,tallbuilding
image_0976,suburb
image_0572,tallbuilding
image_0135,kitchen
image_0574,livingroom
image_0500,mountain
image_0922,mountain
image_0593,coast
image_0430,forest
image_0359,tallbuilding
image_0502,street
image_0162,highway
image_0743,coast
image_0278,street
image_0773,office
image_0923,livingroom
image_0536,bedroom
image_0208,suburb
image_0241,coast
image_0711,street
image_0258,livingroom
image_0365,street
image_0783,insidecity
image_0356,mountain
image_0720,highway
image_1039,tallbuilding
image_0760,office
image_0825,street
image_0706,opencountry
image_0788,office
image_0397,bedroom
image_0232,forest
image_0897,opencountry
image_0400,suburb
image_0661,livingroom
image_0199,tallbuilding
image_0438,street
image_0254,office
image_0644,livingroom
image_0565,tallbuilding
image_0837,forest
image_1014,bedroom
image_0249,suburb
image_0617,opencountry
image_0115,street
image_0519,office
image_1013,office
image_0439,opencountry
image_0015,coast
image_00

image_0065,office
image_0499,livingroom
image_0330,bedroom
image_0698,street
image_0142,opencountry
image_0769,office
image_0374,street
image_0302,coast
image_0105,mountain
image_0856,kitchen
image_0272,opencountry
image_0361,kitchen
image_0781,office
image_0673,suburb
image_0364,forest
image_0809,kitchen
image_0404,kitchen
image_0449,insidecity
image_0089,street
image_0002,suburb
image_0444,highway
image_0568,forest
image_0854,office
image_0159,office
image_0623,tallbuilding
image_0408,kitchen
image_0891,mountain
image_0989,coast
image_0746,livingroom
image_0487,forest
image_0141,tallbuilding
image_0072,bedroom
image_0625,insidecity
image_0214,mountain
image_0867,tallbuilding
image_0010,highway
image_0100,highway
image_0819,highway
image_0489,suburb
image_0966,office
image_0771,bedroom
image_0531,livingroom
image_0657,highway
image_0947,street
image_0469,mountain
image_0774,kitchen
image_0373,street
image_0305,suburb
image_0053,mountain
image_0990,tallbuilding
image_0676,suburb
image_